In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 7982
WEBHOOK_BASE = ""
ADMIN_URL = "http://the_wallet-agent:7981"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)


In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_operation_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_operation_credential


In [4]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': [{'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': '1cb83f3d-380c-4def-8fe6-5640857ee189', 'comment': 'create automated credential exchange', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'fullname', 'value': ''}, {'name': 'age', 'value': '1'}, {'name': 'date', 'value': '1'}, {'name': 'doctorname', 'value': '1'}, {'name': 'complaint', 'value': '1'}, {'name': 'examination results', 'value': '1'}, {'name': 'final treatment', 'value': '1'}, {'name': 'medicine', 'value': '1'}, {'name': 'nextappointment', 'value': '1'}]}, 'cred_def_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:44:default', 'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:MedicalRecords:0.0.1'}, 'auto_remove': True, 'role': 'holder', 'trace': False, 'updated_at': '2023-01-05 10:12:41.941075Z', 'thread_id': 'd8a51a89-4adf-4e27-a389-ab6f41cc2039', 'credential_offer': {'schem

In [5]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

Credential exchange 6736722c-4c3d-4c6b-b0d6-1930940ab374, role: holder, state: offer_received
Being offered: [{'name': 'fullname', 'value': ''}, {'name': 'age', 'value': '1'}, {'name': 'date', 'value': '1'}, {'name': 'doctorname', 'value': '1'}, {'name': 'complaint', 'value': '1'}, {'name': 'examination results', 'value': '1'}, {'name': 'final treatment', 'value': '1'}, {'name': 'medicine', 'value': '1'}, {'name': 'nextappointment', 'value': '1'}]


In [6]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 6736722c-4c3d-4c6b-b0d6-1930940ab374, role: holder, state: request_sent


In [7]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "My Sensitive data")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 6736722c-4c3d-4c6b-b0d6-1930940ab374, role: holder, state: credential_acked


### You can get  credential by your credential id ref(seller notebook)

In [8]:
response = await agent_controller.terminate()
print(response)

None
